In [47]:
import pandas as pd
import numpy as np

In [48]:
df = pd.read_csv('missing.csv')

In [49]:
sector = df.sector.value_counts().tail(106).index.to_list()
df.sector = df.sector.apply(lambda x:np.nan if x in sector else x)
df.dropna(inplace=True)

In [50]:
df.drop(columns={'society_name','rate'},inplace=True)

In [51]:
X = df.drop(columns=['Price'])
y = df['Price']

In [52]:
df.floor.isna().sum()

np.int64(0)

In [53]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [54]:
def feature(score):
    if 0 <= score <= 10:
        return "Low"
    elif 11 <= score < 19:
        return "Medium"
    elif 19 <= score <= 32:
        return "High"
    else:
        return None
def categorize_floor(floor):
    if floor<0:
        return "Basement"
    if 0 <= floor <= 2:
        return "Low Floor"
    elif 3 <= floor <= 10:
        return "Mid Floor"
    elif 11 <= floor <= 51:
        return "High Floor"
    else:
        return None 
def furnish(score):
    if score == 0:
        return "Unfurnished"
    elif 0 < score <= 10:
        return "Low"
    elif 11 <= score < 30:
        return "Medium"
    else:
        return "High"

In [55]:
df['feature_score'] = df['feature_score'].apply(feature)
df['floor'] = df['floor'].apply(categorize_floor)
df['furnish_score'] = df['furnish_score'].apply(furnish)

In [56]:
df.sample(5)

,area,property_type,bedRoom,bathroom,balcony,sector,Price,Study_room,Servant_room,Pooja_room,Other,floor,age,feature_score,furnish_score
5,6269.00,flat,2,2,3,36,0.41,0,0,0,0,Mid Floor,Newly House,Low,Unfurnished
2815,6201.00,flat,3,3,3,95,0.96,0,0,1,0,Mid Floor,Newly House,Low,Unfurnished
3007,1782.24,house,8,8,No,Manesar,1.10,0,0,0,0,Low Floor,Newly House,Low,Unfurnished
480,10567.00,flat,5,5,2,110,4.10,0,1,0,0,Mid Floor,Relatively New,High,Medium
3025,1120.04,house,3,2,2,Sohna,0.85,0,0,0,0,Low Floor,Under Construction,Low,Low


In [57]:
columns_to_encode = ['feature_score','furnish_score','floor','sector','balcony']

In [58]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [59]:
ohe = OneHotEncoder()
ohe.fit(X[['feature_score','furnish_score','floor','sector','balcony']])

,"categories categories: 'auto' or a list of array-like, default='auto'Categories (unique values) per feature:- 'auto' : Determine categories automatically from the training data.- list : ``categories[i]`` holds the categories expected in the ith column. The passed categories should not mix strings and numeric values within a single feature, and should be sorted in case of numeric values.The used categories can be found in the ``categories_`` attribute... versionadded:: 0.20",'auto'
,"drop drop: {'first', 'if_binary'} or an array-like of shape (n_features,), default=NoneSpecifies a methodology to use to drop one of the categories perfeature. This is useful in situations where perfectly collinearfeatures cause problems, such as when feeding the resulting datainto an unregularized linear regression model.However, dropping one category breaks the symmetry of the originalrepresentation and can therefore induce a bias in downstream models,for instance for penalized linear classification or regression models.- None : retain all features (the default).- 'first' : drop the first category in each feature. If only one category is present, the feature will be dropped entirely.- 'if_binary' : drop the first category in each feature with two categories. Features with 1 or more than 2 categories are left intact.- array : ``drop[i]`` is the category in feature ``X[:, i]`` that should be dropped.When `max_categories` or `min_frequency` is configured to groupinfrequent categories, the dropping behavior is handled after thegrouping... versionadded:: 0.21 The parameter `drop` was added in 0.21... versionchanged:: 0.23 The option `drop='if_binary'` was added in 0.23... versionchanged:: 1.1 Support for dropping infrequent categories.",None
,"sparse_output sparse_output: bool, default=TrueWhen ``True``, it returns a :class:`scipy.sparse.csr_matrix`,i.e. a sparse matrix in ""Compressed Sparse Row"" (CSR) format... versionadded:: 1.2 `sparse` was renamed to `sparse_output`",True
,"dtype dtype: number type, default=np.float64Desired dtype of output.",<class 'numpy.float64'>
,"handle_unknown handle_unknown: {'error', 'ignore', 'infrequent_if_exist', 'warn'}, default='error'Specifies the way unknown categories are handled during :meth:`transform`.- 'error' : Raise an error if an unknown category is present during transform.- 'ignore' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will be all zeros. In the inverse transform, an unknown category will be denoted as None.- 'infrequent_if_exist' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will map to the infrequent category if it exists. The infrequent category will be mapped to the last position in the encoding. During inverse transform, an unknown category will be mapped to the category denoted `'infrequent'` if it exists. If the `'infrequent'` category does not exist, then :meth:`transform` and :meth:`inverse_transform` will handle an unknown category as with `handle_unknown='ignore'`. Infrequent categories exist based on `min_frequency` and `max_categories`. Read more in the :ref:`User Guide `.- 'warn' : When an unknown category is encountered during transform a warning is issued, and the encoding then proceeds as described for `handle_unknown=""infrequent_if_exist""`... versionchanged:: 1.1 `'infrequent_if_exist'` was added to automatically handle unknown categories and infrequent categories... versionadded:: 1.6 The option `""warn""` was added in 1.6.",'error'
,"min_frequency min_frequency: int or float, default=NoneSpecifies the minimum frequency below which a category will beconsidered infrequent.- If `int`, categories with a smaller cardinality will be considered infrequent.- If `float`, categories with a smaller cardinality than `min_frequency * n_samples` will be considered infrequent... versionadded:: 1.1 Read more in the :ref:`User Guide `.",None
,"max_catego

In [60]:
column_trans=make_column_transformer(
    (OneHotEncoder(categories=ohe.categories_,drop='first'),columns_to_encode),
    (StandardScaler(), ['bedRoom', 'bathroom', 'area']),
    (OrdinalEncoder(),['property_type','age']),
    remainder='passthrough')

In [61]:
svr = SVR(kernel='rbf')
lr = LinearRegression()

In [62]:
# Creating a pipeline
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('regressor', SVR(kernel='rbf'))
# ])

In [63]:
# Creating a pipeline
pipeline = make_pipeline(column_trans,svr)

In [64]:
# # K-fold cross-validation
# kfold = KFold(n_splits=10, shuffle=True, random_state=42)
# scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [66]:
pipeline.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('svr', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('onehotencoder', ...), ('standardscaler', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the out

In [67]:
y_pred = pipeline.predict(X_test)

In [68]:
from sklearn.metrics import r2_score

In [69]:
r2_score(y_test,y_pred)

0.8041567758107446

In [70]:
# y_pred2 = np.expm1(y_pred)

In [71]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),np.expm1(y_pred))

0.6871923633184249

## Model Selection

In [72]:
# !pip install xgboost

In [73]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA
# Tree-based models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor
)

# Neural Network
from sklearn.neural_network import MLPRegressor

# XGBoost
from xgboost import XGBRegressor


In [74]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [75]:
# scores=[]
# for i in range(1000):
#     X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=i)
#     lr=LinearRegression()
#     pipe=make_pipeline(column_trans,lr)
#     pipe.fit(X_train,y_train)
#     y_pred=pipe.predict(X_test)
#     scores.append(r2_score(y_test,y_pred))

In [76]:
# scores[np.argmax(scores)]

In [77]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [78]:
# model_output = []
for model_name,model in model_dict.items():
    # model_output.append(scorer(model_name, model))
    pipeline = make_pipeline(column_trans,model)
    pipeline.fit(X_train,y_train)
    y_pred = pipeline.predict(X_test)
    print(f" model is {model_name} and r2_score is {r2_score(y_test,y_pred)} and error is {mean_absolute_error(np.expm1(y_test),np.expm1(y_pred))}")

 model is linear_reg and r2_score is 0.7387028009001775 and error is 0.8732019937695268
 model is svr and r2_score is 0.8041567758107446 and error is 0.6871923633184249
 model is ridge and r2_score is 0.7570244375175464 and error is 0.8106342897911593
 model is LASSO and r2_score is -0.00021697241587892435 and error is 1.5827672722146149
 model is decision tree and r2_score is 0.7851642584577025 and error is 0.6158357534246576
 model is random forest and r2_score is 0.83400432308462 and error is 0.5585154945118642
 model is extra trees and r2_score is 0.8500492710552849 and error is 0.5404034934041303
 model is gradient boosting and r2_score is 0.812742683592706 and error is 0.6800879641841685
 model is adaboost and r2_score is 0.4911160535331194 and error is 1.155973383827368
 model is mlp and r2_score is 0.765135532598033 and error is 0.7711350646987277
 model is xgboost and r2_score is 0.8469413275685269 and error is 0.585505298003244


In [79]:
## If i do drop the value and drop 75 sectors
#  model is linear_reg and r2_score is 0.7584810736050396 and error is 0.7514726369863145
#  model is svr and r2_score is 0.16836895163707344 and error is 1.2376096140724708
#  model is ridge and r2_score is 0.7249934152139573 and error is 0.7854042975101148
#  model is LASSO and r2_score is -0.010473202755296951 and error is 1.4627651524441592
#  model is decision tree and r2_score is 0.6978681058255434 and error is 0.6401167115902965
#  model is random forest and r2_score is 0.8220073876250422 and error is 0.5127004307718018
#  model is extra trees and r2_score is 0.829669156374314 and error is 0.4968639261654347
#  model is gradient boosting and r2_score is 0.7948755881552096 and error is 0.6092767573674068
#  model is adaboost and r2_score is 0.30233696914605157 and error is 1.216338827569
# C:\Users\rahul\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
#   warnings.warn(
#  model is mlp and r2_score is 0.7229350073284817 and error is 0.8635005660244532
#  model is xgboost and r2_score is 0.8246756616338907 and error is 0.5343954588521241

In [80]:
# # if i don't drop any other columns
# model is linear_reg and r2_score is 0.7619751238234098 and error is 0.7537248846864684
#  model is svr and r2_score is 0.8122166604327133 and error is 0.5942745808753805
#  model is ridge and r2_score is 0.7669718759946677 and error is 0.7264354663773532
#  model is LASSO and r2_score is -0.01289057680096195 and error is 1.4566084526995169
#  model is decision tree and r2_score is 0.6845399271631469 and error is 0.6892991913746631
#  model is random forest and r2_score is 0.8170405927321499 and error is 0.5260391615100606
#  model is extra trees and r2_score is 0.848414614719087 and error is 0.4862838577130432
#  model is gradient boosting and r2_score is 0.7974936005694702 and error is 0.6113396114048022
#  model is adaboost and r2_score is 0.3884460588050348 and error is 1.1375083994265867
#  model is mlp and r2_score is 0.7774305319945678 and error is 0.6820148920649646
#  model is xgboost and r2_score is 0.836646240927595 and error is 0.4972380800162043

In [81]:
df.sample(2)

,area,property_type,bedRoom,bathroom,balcony,sector,Price,Study_room,Servant_room,Pooja_room,Other,floor,age,feature_score,furnish_score
3522,3600.02,house,5,6,3+,109,9.4,1,1,1,0,Low Floor,Relatively New,High,High
3382,980.99,house,5,4,1,3,0.8,0,0,0,0,Low Floor,Mid,Low,Unfurnished


In [82]:
df.isna().sum()

area             0
property_type    0
bedRoom          0
bathroom         0
balcony          0
sector           0
Price            0
Study_room       0
Servant_room     0
Pooja_room       0
Other            0
floor            0
age              0
feature_score    0
furnish_score    0
dtype: int64

In [83]:
df.to_csv('final.csv',index=False)

In [84]:
df.isna().sum()

area             0
property_type    0
bedRoom          0
bathroom         0
balcony          0
sector           0
Price            0
Study_room       0
Servant_room     0
Pooja_room       0
Other            0
floor            0
age              0
feature_score    0
furnish_score    0
dtype: int64